In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
import cv2
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.models import Sequential, load_model
from keras.optimizers import Adam

In [ ]:
imageGen = ImageDataGenerator(ImageDataGenerator(
        rescale= 1./255.,
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=30,
        horizontal_flip=True,
        brightness_range=(0.5, 1.0)))


In [ ]:
train_set = imageGen.flow_from_directory('/kaggle/input/brain-tumor-mri-dataset/Training',
                                                 target_size = (224,224),
                                                 color_mode = "rgb",
                                                 batch_size=16,
                                                 class_mode = 'binary',
                                                 classes={'glioma': 0., 'meningioma': 1.,'notumor':2,'pituitary':3})
test_set = imageGen.flow_from_directory('/kaggle/input/brain-tumor-mri-dataset/Testing',
                                                 target_size = (224,224),
                                                 color_mode = "rgb",
                                                 batch_size=16,
                                                 class_mode = 'binary',
                                                 classes={'glioma': 0., 'meningioma': 1.,'notumor':2,'pituitary':3})

In [ ]:
train_set.class_indices

In [5]:
#path_of_model = '/kaggle/input/bt-model-weights/model_brain_tumor.h5'
path_of_model = '/kaggle/input/resnet50-model-weights/resnet50_weights_tf_dim_ordering_tf_kernels.h5'

In [6]:
base_model = ResNet50(weights=None)

In [8]:
base_model = load_model(path_of_model)

In [10]:
base_model.trainable = False
model1 = Sequential([
    base_model, 
    Flatten(),
    Dense(256, activation="relu", kernel_initializer='he_normal'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])


In [11]:
model1.compile(optimizer=
              Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 4)                 25893996  
                                                                 
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 256)               1280      
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 25,896,304
Trainable params: 2,308
Non-trainable params: 25,893,996
_________________________________________________________________


In [15]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model_brain_tumor.h5', 
                                                monitor='accuracy', verbose=1, 
                                                mode='max',save_best_only=True)
early = tf.keras.callbacks.EarlyStopping(monitor="accuracy",
                                         mode="max",
                                         restore_best_weights=True,
                                         patience=10)
callbacks_list = [checkpoint,early]

history = model1.fit(train_set,
                     validation_data=test_set,
                     epochs=200,
                     callbacks=callbacks_list)

Epoch 1/200
357/357 [==============================] - ETA: 0s - loss: 0.3714 - accuracy: 0.8810
Epoch 1: accuracy improved from -inf to 0.88095, saving model to model_brain_tumor.h5
357/357 [==============================] - 623s 2s/step - loss: 0.3714 - accuracy: 0.8810 - val_loss: 0.3498 - val_accuracy: 0.8818
Epoch 2/200
357/357 [==============================] - ETA: 0s - loss: 0.3583 - accuracy: 0.8817
Epoch 2: accuracy improved from 0.88095 to 0.88165, saving model to model_brain_tumor.h5
357/357 [==============================] - 579s 2s/step - loss: 0.3583 - accuracy: 0.8817 - val_loss: 0.3439 - val_accuracy: 0.8833
Epoch 3/200
357/357 [==============================] - ETA: 0s - loss: 0.3537 - accuracy: 0.8836
Epoch 3: accuracy improved from 0.88165 to 0.88358, saving model to model_brain_tumor.h5
357/357 [==============================] - 592s 2s/step - loss: 0.3537 - accuracy: 0.8836 - val_loss: 0.3496 - val_accuracy: 0.8802
Epoch 4/200
357/357 [============================

In [16]:
model1.evaluate(train_set)

357/357 [==============================] - 494s 1s/step - loss: 0.3004 - accuracy: 0.8936


[0.3004097640514374, 0.8935574293136597]

In [17]:
model1.evaluate(test_set)

82/82 [==============================] - 113s 1s/step - loss: 0.3396 - accuracy: 0.8810


[0.3396451771259308, 0.8810068368911743]

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
labels = [i: i in train_set.class_indices]

In [ ]:
def predict_image(path):
    img = image.load_img(path,target_size=(224,224))
    img = np.expand_dims(img, axis=0)
    p = model1.predict(img)
    pred = [np.argmax(element) for element in p]
    print (plt.imshow(cv2.imread(path)))
    return labels[pred[0]]

In [ ]:
predict_image('/kaggle/input/brain-tumor-classification-mri/Testing/glioma_tumor/image(10).jpg')